In [ ]:
# import libs
import pandas as pd
import numpy as np
from pandas import read_excel
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import pipeline, preprocessing as ppc, arima
from pmdarima import auto_arima
import pandas.testing as tm
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, make_scorer, mean_squared_error as mse, mean_absolute_error as mae
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

In [ ]:
# merge datasets

# load datasets
dataset1 = 'Session-Details-Summary-20190520 Bauerle.xlsx'
dataset2 = 'Session-Details-Summary-20190520 downtown campus.xlsx'
dataset3 = 'Session-Details-Summary-20190520 1604 campus.xlsx'

# read datasets
df1 = pd.read_excel(dataset1)
df2 = pd.read_excel(dataset2)
df3 = pd.read_excel(dataset3)

# extract colummns to be used
values1 = df1[['Transaction Date (Pacific Time)', 'Energy (kWh)']]
values2 = df2[['Transaction Date (Pacific Time)', 'Energy (kWh)']]
values3 = df3[['Transaction Date (Pacific Time)', 'Energy (kWh)']]

# create a new dataframe with all joined datasets
dataframes = [values1, values2, values3]
join = pd.concat(dataframes)

# save to new csv file called data 
# csv is better output format because of dates
join.to_csv('data.csv', index = False)


In [ ]:
# read new dataset
data = pd.read_csv('data.csv')

# filter desired rows from dataset
data_ranges = [slice(0, 124), slice(134, 220), slice(247, 277), slice(288, 553), slice(704, 867), slice(873, 1015)]
data = pd.concat(data.iloc[x, :] for x in data_ranges)

# rename columns and set index
data.set_index('Transaction Date (Pacific Time)', inplace= True)
data.index.name = 'date'
data.columns = ["power"]

#convert from energy (kWh) to power (kW) 
data.index = pd.to_datetime(data.index)
data = data[data.index.dayofweek < 5].sort_values(by="date")
print(data.head(20))
data.describe()
# make sure all values are numeric
data = data.astype('float32')

In [ ]:
# summary
data.describe()

Summary table indicates that most of the dataset is centered around 6 and dispersion is also 6, that means that maximum value can be considered anomaly in data.

In [ ]:
# visualize time series 

fig = plt.figure(figsize=(18,16))
ax = fig.add_subplot(5,1,1)
ax.plot(data,linewidth=1)
ax.set_title('Load power resampled over day')
ax.tick_params(axis='both', which='major')
plt.xlabel('Time')
plt.ylabel('Load Energy (kW)')
plt.show()

In [ ]:
# we want to look at p-value to know if time series is stationary
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

print(adf_test(data))

P-value indicates that series is stationary

In [ ]:
# seasonal decomposition will help to see we have seasonal pattern or not!
a = seasonal_decompose(data, model = "add", period=30)

In [ ]:
plt.figure(figsize = (20,8))
a.plot();

Clearly we have monthly seasonality, some extreme residuals can be seen, which also indicate that there might be outlier in the data. trend is not consistent.

In [ ]:
# split data with 80/20 ratio: first split is train and other test
# sequence has to be maintained because of time series data

N = int(len(data)*0.8)
train_data, test_data = data[:N], data[N:]
scaler = MinMaxScaler()
scaler.fit(train_data)
train = scaler.transform(train_data)
test = scaler.transform(test_data)

In [ ]:
print("Train Shape", train.shape, "Test shape", test.shape)

The arima code is written with guidance from this page:

http://alkaline-ml.com/pmdarima/1.5.1/auto_examples/example_pipeline.html#sphx-glr-auto-examples-example-pipeline-py

In [ ]:
# Let's create a pipeline with multiple stages... power dataset seems to be showing monthly seasonality
#so we'll include a FourierFeaturizer so we can fit it without seasonality
# we have chosen 6 month as seasonal pattern, which will be divided in 75 variable

pipe = pipeline.Pipeline([
    ("fourier", ppc.FourierFeaturizer(m=150, k=75)),
    ("arima", arima.AutoARIMA(stepwise=True, trace=1, error_action="ignore",
                              seasonal=False,  # because we use Fourier
                              suppress_warnings=True))
])


In [ ]:
# fit arima model
pipe.fit(train)
print("Model fit:")
print(pipe)

# We can compute predictions the same way we would on a normal ARIMA object:
preds, conf_int = pipe.predict(n_periods=20, return_conf_int=True)
print("\nForecasts:")
print(preds)


# Visualize goodness of fit
in_sample_preds, in_sample_confint = \
    pipe.predict_in_sample(exogenous=None, return_conf_int=True)

n_train = train.shape[0]


# We can also call `update` directly on the pipeline object, which will update
# the intermittent transformers, where necessary:
newly_observed, still_test = test[:100], test[100:]
pipe.update(newly_observed, maxiter=100)

# Calling predict will now predict from newly observed values
new_preds = pipe.predict(still_test.shape[0])
print(new_preds)


In [ ]:
# Let's take a look at the actual vs. the predicted values:
fig, axes = plt.subplots(3, 1, figsize=(12, 8))
fig.tight_layout()

x0 = np.arange(n_train)
axes[0].plot(x0, train, alpha=0.75)
axes[0].scatter(x0, in_sample_preds, alpha=0.4, marker='x', color="red")
axes[0].set_title('Actual train samples vs. in-sample predictions')
axes[0].set_xlim((0, x0.shape[0]))

x1 = np.arange(n_train + preds.shape[0])
axes[1].plot(x1[:n_train], train, alpha=0.75)
axes[1].scatter(x1[n_train:], preds, alpha=0.4, marker='o')
axes[1].scatter(x1[n_train:], test[:preds.shape[0]], alpha=0.4, marker='x')
axes[1].fill_between(x1[n_train:], conf_int[:, 0], conf_int[:, 1],
                     alpha=0.1, color='b')
axes[1].set_title('Actual test samples vs. forecasts')
axes[1].set_xlim((0, data.shape[0]))

x2 = np.arange(data.shape[0])
n_trained_on = n_train + newly_observed.shape[0]

axes[2].plot(x2[:n_train], train, alpha=0.75)
axes[2].plot(x2[n_train: n_trained_on], newly_observed, alpha=0.75, c='orange')
axes[2].scatter(x2[n_trained_on:], still_test, alpha=0.4, marker='x', c='red')
axes[2].set_title('Actual test samples vs. forecasts')
axes[2].set_xlim((0, data.shape[0]))

plt.show()

In [ ]:
arima_pr = pipe.predict(test.shape[0])

In [ ]:
# Arima Predictions for test dataset and transformation
arima_pr = pipe.predict(test.shape[0])
arima_predictions =scaler.inverse_transform(arima_pr.reshape(test.shape[0],1))

In [ ]:
# now merge with test data
test_copy = test_data["power"].copy().to_frame()
test_copy["Arima Predictions"] = arima_predictions

In [ ]:
# plot
test_copy.plot(figsize=(20,8))

Not very impressive!

# LSTM model

## 1 Step LSTM

In [ ]:

def build_model(n_neurons=30, input_shape=[1], optimizer='adam'):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(input_shape))
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model


# define the grid search parameters
optimizers = ['Adam','Nadam', 'Adagrad']
neurons = [200, 400, 600, 800, 1000]
batch_size = [10,20,30]
epoch = [500, 1000]
keras_reg = KerasRegressor(build_model)
params_distrib = dict(batch_size=batch_size, n_neurons=neurons, optimizer=optimizers,
                     epochs=epoch)
rnd_search_cv = RandomizedSearchCV(keras_reg, params_distrib, cv=3,
                                   scoring="r2")
grid_result = rnd_search_cv.fit(train, train, epochs=50, 
                    validation_data=(test, test))

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f (Standard Deviation: %f) with: %r" % (mean, stdev, param))

In [ ]:
# plot accuracy
plt.plot(grid_result.best_estimator_.model.history.history['accuracy'])
plt.plot(grid_result.best_estimator_.model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# plot loss
plt.plot(grid_result.best_estimator_.model.history.history['loss'])
plt.plot(grid_result.best_estimator_.model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
lstm_1_pr = grid_result.predict(test)
lstm1_predictions = scaler.inverse_transform(lstm_1_pr.reshape(test.shape[0],1))

In [ ]:
train2 = train.reshape(int(len(train)/5), 5)
test2 = test.reshape(int(len(test)/5), 5)

##  Step LSTM

In [ ]:
def build_model2(n_neurons=30, input_shape=[5], optimizer='adam'):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(input_shape))
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(5))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model


keras_reg1 = KerasRegressor(build_model2)
params_distrib1 = dict(batch_size=batch_size, n_neurons=neurons, optimizer=optimizers,
                     epochs=epoch)
rnd_search_cv1 = RandomizedSearchCV(keras_reg1, params_distrib1, cv=3,
                                   scoring="r2")
grid_result2 = rnd_search_cv1.fit(train2, train2, epochs=50, 
                    validation_data=(test2, test2))

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))
means2 = grid_result2.cv_results_['mean_test_score']
stds2 = grid_result2.cv_results_['std_test_score']
params2 = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means2, stds2, params2):
    print("Mean: %f (Standard Deviation: %f) with: %r" % (mean, stdev, param))

In [ ]:
# plot accuracy
plt.plot(grid_result2.best_estimator_.model.history.history['accuracy'])
plt.plot(grid_result2.best_estimator_.model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# plot loss
plt.plot(grid_result2.best_estimator_.model.history.history['loss'])
plt.plot(grid_result2.best_estimator_.model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
lstm_2_pr = grid_result2.predict(test2)
lstm2_predictions = scaler.inverse_transform(lstm_2_pr.reshape(test.shape[0],1))

In [ ]:
train3 = train.reshape(int(len(train)/15), 15)
test3 = test.reshape(int(len(test)/15),15)

## 15 Step LSTM

In [ ]:
def build_model3(n_neurons=30, input_shape=[15], optimizer='adam'):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(input_shape))
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(15))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model


keras_reg2 = KerasRegressor(build_model3)
params_distrib2 = dict(batch_size=batch_size, n_neurons=neurons, optimizer=optimizers,
                     epochs=epoch)
rnd_search_cv2 = RandomizedSearchCV(keras_reg2, params_distrib1, cv=3,
                                   scoring="r2")
grid_result3 = rnd_search_cv2.fit(train3, train3, epochs=50, 
                    validation_data=(test3, test3))

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result3.best_score_, grid_result3.best_params_))
means3 = grid_result3.cv_results_['mean_test_score']
stds3 = grid_result3.cv_results_['std_test_score']
params3 = grid_result3.cv_results_['params']
for mean, stdev, param in zip(means3, stds3, params3):
    print("Mean: %f (Standard Deviation: %f) with: %r" % (mean, stdev, param))

In [ ]:
lstm_3_pr = grid_result3.predict(test3)
lstm3_predictions = scaler.inverse_transform(lstm_3_pr.reshape(test.shape[0],1))

In [ ]:
# plot accuracy
plt.plot(grid_result3.best_estimator_.model.history.history['accuracy'])
plt.plot(grid_result3.best_estimator_.model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# plot loss
plt.plot(grid_result3.best_estimator_.model.history.history['loss'])
plt.plot(grid_result3.best_estimator_.model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
test_copy["1_step LSTM_Predictions"] = lstm1_predictions
test_copy["5_step LSTM_Predictions"] = lstm2_predictions
test_copy["15_step LSTM_Predictions"] = lstm3_predictions

In [ ]:
# plot
test_copy.plot(figsize=(20,12))

LSTM Predictions looks to be more accurate!

## Plot all lstm models

In [ ]:
# One step and multi step LSTM forecasting graph
fig = plt.figure(dpi=700,figsize=(12, 8))
aa=test_copy.index
#linestyles = ['-', '--', '-.', ':']
plt.plot(aa, test_copy["power"], "b", linestyle='dashed', marker='*', label="Actual")

plt.plot(aa, test_copy["1_step LSTM_Predictions"], 'red', linestyle='dotted', marker = 's',
         label="1 Step LSTM predictions")

plt.plot(aa, test_copy["5_step LSTM_Predictions"], 'g', marker = '^', linestyle=':',
         label="5 Step LSTM Predictions")

plt.plot(aa, test_copy["15_step LSTM_Predictions"], 'gray', marker = '>',
         label="15 Step LSTM Predictions")

sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Global active power', size=20)
plt.xlabel('Date', size=15)
plt.title("LSTM Predictions With different Steps", fontsize=22, color="black")
plt.legend(fontsize=15)
plt.show()

In [ ]:
# ARIMA forecasting

fig = plt.figure(dpi=700,figsize=(12, 8))
aa=test_copy.index
plt.plot(aa, test_copy["power"].to_numpy(), marker='*', label="Actual")
plt.plot(aa, test_copy["Arima Predictions"].to_numpy(), 'r', marker = 'o',
         label="Arima Predictions")

plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Global active power', size=20)
plt.xlabel('Date', size=15)
plt.title("ARIMA Predictions", fontsize=22, color="black")
plt.legend(fontsize=15)
plt.show();

In [ ]:
ts_desc = test_copy.describe()

In [ ]:
# Calculate Error metrics
def error_metric(b, a = test_copy["power"]):
    ms = mse(a,b)
    ma = mae(a, b)
    r2 = r2_score(a, b)
    return (ms, ma, r2)
    

In [ ]:
mse_ar, mae_ar, r_2_ar = error_metric(test_copy["Arima Predictions"])
mse1, mae1, r_21 = error_metric(test_copy["1_step LSTM_Predictions"])
mse2, mae2, r_22 = error_metric(test_copy["5_step LSTM_Predictions"])
mse4, mae4, r_24 = error_metric(test_copy["15_step LSTM_Predictions"])


In [ ]:
data = {'Power': ['', '', ''],
        'Arima': [mse_ar, mae_ar, r_2_ar],
        '1 Step LSTM': [mse1, mae1, r_21],
       '5 Step LSTM': [mse2, mae2, r_22],
       '15 Step LSTM' : [mse4, mae4, r_24]}

error_df = pd.DataFrame.from_dict(data, orient='index', columns=['MSE', 'MAE', "R-Square"])

In [ ]:
ts_desc.columns = error_df.T.columns

In [ ]:
ts_desc

In [ ]:
# these lines can be removed, if not needed!
error_df["Mean"] = ts_desc.T["mean"]
error_df["SD"] = ts_desc.T["std"]

In [ ]:
error_df

In [ ]:
ts_desc

In [ ]:
grid_result.cv_results_["std_test_score"]

In [ ]:
result1 = pd.concat([pd.DataFrame(grid_result.cv_results_["params"]),
                     pd.DataFrame(grid_result.cv_results_["mean_test_score"], columns=["Accuracy"]),
                     pd.DataFrame(grid_result.cv_results_["std_test_score"], columns=["Error"])],axis=1)

In [ ]:
result1

In [ ]:
result2 = pd.concat([pd.DataFrame(grid_result2.cv_results_["params"]),
                     pd.DataFrame(grid_result2.cv_results_["mean_test_score"], columns=["Accuracy"]),
                     pd.DataFrame(grid_result2.cv_results_["std_test_score"], columns=["Error"])],axis=1)

In [ ]:
result3 = pd.concat([pd.DataFrame(grid_result3.cv_results_["params"]),
                     pd.DataFrame(grid_result3.cv_results_["mean_test_score"], columns=["Accuracy"]),
                     pd.DataFrame(grid_result3.cv_results_["std_test_score"], columns=["Error"])],axis=1)

In [ ]:
step_results = pd.concat([result1, result2, result3], axis=1, keys=['1 Step', '5 Step', '15 Step'])

In [ ]:
r = step_results.stack(0).reset_index()

In [ ]:
r.groupby(["level_1", "optimizer"])["Accuracy"].plot(figsize=(16,12), legend=True)
plt.show()

In [ ]:
r.groupby(["level_1", "optimizer"])["Accuracy"].plot(figsize=(16,12), legend=True)
plt.show()

In [ ]:
# lets filter based on more than 95% accuracy
rr = r[r["Accuracy"] > 0.90].iloc[:, 1:]

In [ ]:
rr